# Deploying an ensemble model with custom application logic

In the last notebook, we learned how to deploy two models that don't interact.
In this notebook, we'll learn how to compose these models together into an *ensemble* model.
We'll use custom logic expressed directly in Python to determine how to compose the models.

First, we'll start with a simple version that takes half its results from the plot-based recommender and half from the color-based recommender.
Once we have that working, we'll try a more complex version that learns *online* how much of each to return, based on the particular user's preferences.
For example, a user that prefers recommendations based on plot will receive more and more of these with each new request.

## If you haven't finished notebooks 1 and/or 2 yet:

Evaluate the next cell to fill out the missing movie palettes in the local database and deploy a plot and color recommender.
This will let you do this notebook without having finished the first two yet.

In [ ]:
%%bash
# These commands might take several seconds to run.
bash run_1.sh
bash run_2.sh

## Connect to Ray Serve

First let's set up a connection to the Ray cluster and Serve instance that we started in the last notebook.

In [ ]:
import ray
from ray import serve
from util import MOVIE_IDS
import requests

ray.init(address="auto", ignore_reinit_error=True)
client = serve.connect()

# Some helper methods copied from the previous notebook to ping the color and plot backends.
def send_color_request(movie_id):
    color_handle = client.get_handle("color")
    return ray.get(color_handle.remote(liked_id=MOVIE_IDS[0]))

def send_plot_request(movie_id):
    plot_handle = client.get_handle("plot")
    return ray.get(plot_handle.remote(liked_id=MOVIE_IDS[0]))

## Deploying a static ensemble model

Let's see how we can use the `choose_ensemble_results` helper function to return the right distribution of results from the plot and color recommenders.
This function has 3 return values:

1. The normalized distribution of "color" and "plot" recommendations, computed from the weights given as the first argument.
2. A dict that maps the recommendation type ("color" or "plot") to a list of recommendations of that type that were picked.
3. A list of the concatenated results from the "color" and "plot" recommenders that were given as the second argument. This list is picked based on the normalized distribution.

**Task:** Try playing with the helper function to understand how it works. You can modify the weights given in the first argument and/or the recommendations given in the second argument.

In [ ]:
from util import choose_ensemble_results

color_recs = send_color_request(MOVIE_IDS[0])
plot_recs = send_plot_request(MOVIE_IDS[0])
# Weight color and plot recommendations equally.
choose_ensemble_results({"color": 1, "plot": 1}, {"color": color_recs, "plot": plot_recs})

**Task:** Let's try writing a static ensemble model that fetches results from the color- and plot-based recommenders and combines them with this helper function. Once you've finished the implementation, evaluate the following two cells to deploy the ensemble model and test it out. Here is a skeleton to get you started:

In [ ]:
class ComposedModel:
    def __init__(self):
        # Get handles to the two underlying models.
        client = serve.connect()
        self.color_handle = client.get_handle("color")
        self.plot_handle = client.get_handle("plot")

    def __call__(self, request):
        # TODO: Call the two models and get their predictions.
        # Tip: Check out the implementations of `send_color_request` and `send_plot_request`
        # to see how to use `self.color_handle` and `self.plot_handle` to get the recommendations.

        # TODO: Combine the results using choose_ensemble_results.
        #distribution, impressions, chosen = choose_ensemble_results(...)

        return {
#             "dist": distribution,
#             "ids": chosen,
#             "sources": {
#                 i["id"]: source
#                 for source, impression in impressions.items()
#                 for i in impression
#             }
        }

In [ ]:
# Delete the ensemble endpoint and backend, if they exist.
# We wrap it in a try-except block since they shouldn't exist on the first evaluation.
try:
    client.delete_endpoint("ensemble")
    client.delete_backend("ensemble:v0")
except:
    pass

# Create the ensemble backend and endpoint.
# Tip! You can run this cell again if you need to debug the ComposedModel code.
client.create_backend("ensemble:v0", ComposedModel)
client.create_endpoint("ensemble", backend="ensemble:v0", route="/rec/ensemble")

In [ ]:
def send_ensemble_request(movie_id, session_key=None):
    r = requests.get("http://localhost:8000/rec/ensemble", params={"liked_id": MOVIE_IDS[0],
                                                                   "session_key": session_key})
    if r.status_code == 200:
        return r.json()
    print(r.text)

send_ensemble_request(MOVIE_IDS[0])

## Deploying a custom ensemble model

Now let's try shifting the distribution based on what the user has selected before!
This will involve saving some session state for each user.
The session state will consist of the source ("plot" or "color") for every recommendation made to the user so far, as well as the number of color- and plot-based recommendations that the user previously liked.

On each request to the ensemble model, we'll include a session key that is unique per user which can be used to look up the session state.
Each time the user selects a movie recommendation that they like, we'll look up the source of that recommendation, if we have it.
Then, we'll increment the appropriate count in their session state, either "plot" or "color".

To store each user's session state, we'll use a Ray *actor*.
Just as Ray *tasks* extend the familiar concept of Python *functions*, Ray *actors* extend the concept of Python *classes*.
Whereas tasks are *stateless*, actors are *stateful*, meaning that they keep their local state from task to task.

We'll use a predefined `ImpressionStore` class to store the mapping from session key to session state.
Let's take a look at how this works.

### Reading and writing from the `ImpressionStore` actor

First, we'll create an `ImpressionStore` actor to store each user's session state and show how to extract the user's distribution based on a movie that the test user just liked.
Just like with non-actor tasks, we can submit a task to the actor by specifying a method name and the `.remote()` suffix.
The task returns an `ObjectRef`, whose value we can get using `ray.get()`.

Since we haven't made any recommendations yet, the actor won't have a recommendation source for the movie liked by the user, so it will just return an empty dictionary.
This means that the actor knows nothing about the number of color- versus plot-based recommendations that the user prefers.

In [ ]:
from util import ImpressionStore

# Create an actor to store the session state.
impression_store = ImpressionStore.remote()
test_session_key = "session"

# Initially, we have no information about the test user's preferred distribution.
print("Initial distribution:",
      dict(ray.get(impression_store.model_distribution.remote(test_session_key, MOVIE_IDS[0]))))

Next we'll mimic getting some results from the color and plot recommenders and show to update the `ImpressionStore` actor's state with the source of the recommendations.

In [ ]:
# Mimic getting some results from the color and plot recommenders.
color_recs = send_color_request(MOVIE_IDS[0])
plot_recs = send_plot_request(MOVIE_IDS[0])

# impressions is a dictionary mapping the recommendation source ("color" or "plot") to a
# list of movies returned by that recommender.
_, impressions, recs = choose_ensemble_results({"color": 1, "plot": 1}, {"color": color_recs, "plot": plot_recs})

# Record the source of recommendations made to the test user.
# We don't need to call ray.get this time because we're just updating the actor's state
# and we don't need to wait for the reply.
impression_store.record_impressions.remote(test_session_key, impressions)

print("Initial recommendations for movie {}:".format(MOVIE_IDS[0]))
recs

Now that we've recorded some recommendations in the actor, we can see what happens to the user's distribution when we choose one of these movies.

**Task:** Understand how the user's distribution shifts based on what movies they've liked so far.
1. Evaluate the next cell a couple times and compare the output to the output above. What are the differences?
2. Modify the code to shift the test user's distribution towards plot-based recommendations.

In [ ]:
# Mimic the user selecting a recommendation based on color.
distribution = dict(ray.get(impression_store.model_distribution.remote(test_session_key, color_recs[0]["id"])))
# Now the distribution will assign color recommendations a weight of 1.
new_distribution, _, recs = choose_ensemble_results(distribution, {"color": color_recs, "plot": plot_recs})
print(
    "Distribution after recording the user's preference", new_distribution)

print("Recommendations for movie {} after a user's click:".format(MOVIE_IDS[0]))
recs

### Deploying the `ImpressionStore` with Ray Serve.

Now let's integrate the `ImpressionStore` code with the ensemble model that we deployed earlier.

**Task:**
1. Modify the `CustomComposedModel` skeleton below to use the impression store. We've already instantiated the actor for you in the constructor. Modify the `__call__` method to:
  - Get the user's current distribution from the impression store actor.
  - Update the impression store actor with the new recommendations made for that user.

2. Once you've finished these steps, evaluate the following cell to deploy the new backend.
The following cell generates some test requests that chooses recommendations based on color, so that you can see how the distribution changes with each selection.

In [ ]:
class CustomComposedModel:
    def __init__(self):
        # Get handles to the two underlying models.
        client = serve.connect()
        self.color_handle = client.get_handle("color")
        self.plot_handle = client.get_handle("plot")
        
        # Instantiate an impression store actor.
        self.impression_store = ImpressionStore.remote()

    def __call__(self, request):
        session_key = request["session_key"]
        
        # TODO: Call the two models and get their predictions.
        
        # TODO: Get the user's current distribution from the impression store actor.

        # TODO: Combine the results using choose_ensemble_results.Select which results to send to the user based on their clicks.
        #distribution, impressions, chosen = choose_ensemble_results(...)
        
        # TODO: Update the impression store actor with the sources of the recommendations
        # returned in `chosen`.

        return {
#             "dist": distribution,
#             "ids": chosen,
#             "sources": {
#                 i["id"]: source
#                 for source, impression in impressions.items()
#                 for i in impression
#             }
        }

In [ ]:
# Delete the ensemble endpoint and backend, if they exist.
# We wrap it in a try-except block since they shouldn't exist on the first evaluation.
client.delete_endpoint("ensemble")
client.delete_backend("ensemble:v0")

# Create the ensemble backend and endpoint.
# Tip! You can run this cell again if you need to debug the ComposedModel code.
client.create_backend("ensemble:v0", ComposedModel)
client.create_endpoint("ensemble", backend="ensemble:v0", route="/rec/ensemble")

In [ ]:
response = send_ensemble_request(MOVIE_IDS[0], session_key=test_session_key)
print("Initial recommendation", response)
for movie_id, source in response.items():
    if source == "color":
        break
response = send_ensemble_request(movie_id, session_key=test_session_key)
print("Recommendation after selecting a color recommendation:", response)

## Thanks for joining us today! If you...:

- ...have some time, we'd love to hear your feedback! Please fill out this 1-minute [survey](https://forms.gle/yagdwutzCBrGcmZF7).

- ...want to hear more about Ray and Ray Serve, visit some of these resources:

    - Join the [Ray slack](https://forms.gle/9TSdDYUgxYs8SA9e8)
    - Visit the [GitHub project](https://github.com/ray-project/ray) (more resources linked here)
    - Read the [Ray](https://docs.ray.io/en/latest/index.html) and [Ray Serve](https://docs.ray.io/en/latest/serve/index.html) docs